In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib as mpl ; mpl.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt ; plt.rcParams['font.family'] = 'AppleGothic'
# ----- Our own modules
import variable as var
import Preprocessing as pre
import modeling as md

In [28]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# --------------- Train Set [전처리], [변수추가] ---------------
train = pre.Preprocessing(train, return_train=True) # [전처리]
train = var.gen_size(train) # [변수추가] 소형세대, 중형세대, 대형세대
train = var.code_name(train) # [변수추가] 단지명, 주소 (위도, 경도는 결측값 대체전까지 보류)
train = var.major_voting(train, '자격유형') # [변수추가] 자격유형_major
train = var.major_voting(train, '공급유형') # [변수추가] 공급유형_major
train = var.weigted_fee(train) # [변수추가] 가중평균임대료
train = var.total_member(train) # [변수추가] 총입주민수
train = train.drop(['자격유형','공급유형'], axis=1)
train = train.drop(['임대료', '임대보증금'],axis=1)
train = train.drop(['전용면적','전용면적별세대수'],axis=1)

# train = pre.dummy_train(train,'자격유형_major') # 자격유형_major OneHotEncoding(Test에 없어도 자동변환)
#train = train.loc[:,~train.columns.str.contains('|'.join(['(남자)','(여자)']),regex=True)] #Warning(str.contains)

# --------------- Test Set [전처리], [변수추가] ---------------
test = pre.Preprocessing(test, return_train=False)
test = var.gen_size(test) # [변수추가] 소형세대, 중형세대, 대형세대
test = var.code_name(test) # [변수추가] 단지명, 주소 (위도, 경도는 결측값 대체전까지 보류)
test = var.major_voting(test, '자격유형') # [변수추가] 자격유형_major
test = var.major_voting(test, '공급유형') # [변수추가] 공급유형_major
test = var.weigted_fee(test) # [변수추가] 가중평균임대료
test = var.total_member(test) # [변수추가] 총입주민수
test = test.drop(['자격유형','공급유형'], axis=1)
test = test.drop(['임대료', '임대보증금'],axis=1)
test = test.drop(['전용면적','전용면적별세대수'],axis=1)

train,test = pre.dummy(train,test,'자격유형_major') # 자격유형_major OneHotEncoding(Test에 없어도 자동변환)

In [29]:
train['고령인구비율'] = train[['70대(여자)','70대(남자)','80대(여자)','80대(남자)','90대(여자)','90대(남자)','100대(여자)','100대(남자)']].apply(np.sum, axis=1)
train = train.loc[:,~train.columns.str.contains('|'.join(['(남자)','(여자)']),regex=True)] #Warning(str.contains)
train['세대당가구원'] = train['총입주민수']/train['총세대수']
train = train.drop(['중형세대','대형세대'], axis=1)
test['고령인구비율'] = test[['70대(여자)','70대(남자)','80대(여자)','80대(남자)','90대(여자)','90대(남자)','100대(여자)','100대(남자)']].apply(np.sum, axis=1)
test = test.loc[:,~test.columns.str.contains('|'.join(['(남자)','(여자)']),regex=True)] #Warning(str.contains)
test['세대당가구원'] = test['총입주민수']/test['총세대수']
test = test.drop(['중형세대','대형세대'], axis=1)
test = test.fillna(1.5) # 총입주민수 NA 존재!
train = train.fillna(1.5)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [30]:
md.modeling(train, 'RF')

122.38528925619833

In [34]:
md.submit(train,test)

,code,num
0,C1003,256.70
1,C1006,186.91
2,C1016,682.97
3,C1019,288.08
4,C1030,46.74
...,...,...
145,C2688,90.29
146,C2691,1062.37
147,C2675,0.00
148,C2335,0.00
